# 🎨 Fooocus Batch Generator - Wildcard Módszer

Ez a notebook **automatikusan generálja** a képeket TXT fájlokból **wildcard** használatával.

## 📋 Működés

1. TXT fájlokat **wildcard** fájlokká alakítja
2. Fooocus-t elindítja
3. **Gradio API**-n keresztül generálja a képeket
4. Automatikusan letölti az eredményeket

## 🚀 Használat

Futtasd sorban az összes cellát!

## 1️⃣ Fooocus Telepítése

In [ ]:
import os

# Fooocus klónozása
if not os.path.exists("/content/Fooocus"):
    print("📥 Fooocus letöltése...")
    !git clone https://github.com/lllyasviel/Fooocus.git

# Függőségek
!apt-get -y install -qq aria2
!pip install pygit2==1.15.1 gradio_client==0.8.1 -q

print("✓ Telepítés kész!")

## 2️⃣ TXT Fájlok Feltöltése

In [ ]:
from google.colab import files
import shutil

# Prompts mappa
prompts_dir = "/content/prompts"
os.makedirs(prompts_dir, exist_ok=True)

print("📤 Töltsd fel a TXT fájljaidat:")
uploaded = files.upload()

# Fájlok áthelyezése
for filename in uploaded.keys():
    if filename.endswith('.txt'):
        shutil.move(filename, f"{prompts_dir}/{filename}")
        print(f"✓ {filename}")

print("\n✓ Feltöltés kész!")

## 3️⃣ Wildcard Fájlok Létrehozása

In [ ]:
from pathlib import Path

# Wildcard mappa
wildcard_dir = "/content/Fooocus/wildcards"
os.makedirs(wildcard_dir, exist_ok=True)

# TXT -> Wildcard konverzió
txt_files = list(Path(prompts_dir).glob("*.txt"))

print(f"📝 {len(txt_files)} fájl konvertálása wildcard-okká...\n")

wildcard_mapping = {}

for txt_file in txt_files:
    # Wildcard név (fájlnév .txt nélkül)
    wildcard_name = txt_file.stem
    wildcard_path = f"{wildcard_dir}/{wildcard_name}.txt"
    
    # Másolás
    shutil.copy(txt_file, wildcard_path)
    
    # Promptok száma
    with open(txt_file, 'r', encoding='utf-8') as f:
        prompts = [line.strip() for line in f if line.strip() and not line.startswith('#')]
    
    wildcard_mapping[wildcard_name] = len(prompts)
    print(f"✓ __{wildcard_name}__ → {len(prompts)} prompt")

print(f"\n✓ {len(wildcard_mapping)} wildcard létrehozva!")

## 4️⃣ Beállítások

In [ ]:
import json

# ========== BEÁLLÍTÁSOK ==========

PERFORMANCE = "Extreme Speed"  # Speed, Quality, Extreme Speed
ASPECT_RATIO = "1344*704"      # 1344*704, 1152*896, stb.
STYLES = ["SAI Comic Book"]    # Stílusok
MODEL = "juggernautXL_v8Rundiffusion.safetensors"
OUTPUT_FORMAT = "png"

# Preset mentése
preset_path = "/content/Fooocus/presets/batch_auto.json"

settings = {
    "default_performance": PERFORMANCE,
    "default_aspect_ratio": ASPECT_RATIO,
    "default_styles": STYLES,
    "default_model": MODEL,
    "default_output_format": OUTPUT_FORMAT
}

with open(preset_path, 'w') as f:
    json.dump(settings, f, indent=4)

print("✓ Beállítások:")
print(f"  Performance: {PERFORMANCE}")
print(f"  Felbontás: {ASPECT_RATIO}")
print(f"  Stílus: {STYLES}")

## 5️⃣ Fooocus Indítása (háttérben)

In [ ]:
import subprocess
import time
import requests

os.chdir("/content/Fooocus")

print("🚀 Fooocus indítása...")

# Háttérben indítás
process = subprocess.Popen(
    ['python', 'entry_with_update.py', '--share', '--always-high-vram', '--preset', 'batch_auto'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Várakozás, amíg a szerver elindul
print("⏳ Várakozás a szerver indulására...")
max_wait = 300  # 5 perc max
start_time = time.time()
server_ready = False

while time.time() - start_time < max_wait:
    try:
        response = requests.get("http://127.0.0.1:7865", timeout=2)
        if response.status_code == 200:
            server_ready = True
            break
    except:
        pass
    
    elapsed = int(time.time() - start_time)
    print(f"  Várakozás... ({elapsed}s)", end='\r')
    time.sleep(5)

if server_ready:
    print(f"\n✓ Fooocus elindult! ({int(time.time() - start_time)}s)")
else:
    print("\n❌ Timeout! A szerver nem indult el.")
    print("Próbáld újra futtatni ezt a cellát.")

## 6️⃣ AUTOMATIKUS BATCH GENERÁLÁS

In [ ]:
from gradio_client import Client
import time

# Gradio client
print("🔌 Csatlakozás Fooocus-hoz...")

try:
    client = Client("http://127.0.0.1:7865")
    print("✓ Csatlakozva!\n")
except Exception as e:
    print(f"❌ Csatlakozási hiba: {e}")
    print("\nEllenőrizd, hogy az 5. cella sikeresen lefutott-e!")
    print("Ha nem, futtasd újra az 5. cellát.")
    raise

# ========== GENERÁLÁS ==========

total_generated = 0

for wildcard_name, prompt_count in wildcard_mapping.items():
    print(f"{'='*60}")
    print(f"🎨 Wildcard: __{wildcard_name}__")
    print(f"{'='*60}")
    print(f"Promptok: {prompt_count}\n")
    
    # Batch-enként generálás (max 32)
    batch_size = 32
    batches = (prompt_count + batch_size - 1) // batch_size
    
    for batch_num in range(batches):
        start_idx = batch_num * batch_size
        end_idx = min(start_idx + batch_size, prompt_count)
        current_batch_size = end_idx - start_idx
        
        print(f"\n📦 Batch {batch_num + 1}/{batches} ({current_batch_size} kép)")
        
        # Prompt wildcard-dal
        prompt = f"__{wildcard_name}__"
        
        try:
            # Gradio API hívás
            result = client.predict(
                prompt=prompt,
                negative_prompt="",
                style_selections=STYLES,
                performance_selection=PERFORMANCE,
                aspect_ratios_selection=ASPECT_RATIO,
                image_number=current_batch_size,
                output_format=OUTPUT_FORMAT,
                image_seed=-1,
                seed_random=True,
                api_name="/generate"
            )
            
            total_generated += current_batch_size
            print(f"  ✓ {current_batch_size} kép generálva")
            
        except Exception as e:
            print(f"  ✗ Hiba: {e}")
        
        # Kis szünet batch-ek között
        if batch_num < batches - 1:
            time.sleep(2)
    
    print(f"\n✓ __{wildcard_name}__ kész!\n")

print(f"\n{'='*60}")
print(f"🎉 GENERÁLÁS BEFEJEZVE!")
print(f"{'='*60}")
print(f"Összesen: {total_generated} kép")

## 7️⃣ Képek Letöltése

In [ ]:
import shutil
from pathlib import Path

output_dir = "/content/Fooocus/outputs"

if os.path.exists(output_dir):
    print("📦 Tömörítés...")
    shutil.make_archive('/content/generated_images', 'zip', output_dir)
    
    # Statisztika
    images = list(Path(output_dir).glob('**/*.png')) + list(Path(output_dir).glob('**/*.jpg'))
    total_size = sum(img.stat().st_size for img in images)
    
    print(f"\n📊 Statisztika:")
    print(f"  Képek: {len(images)}")
    print(f"  Méret: {total_size / 1024 / 1024:.2f} MB")
    
    print("\n⬇️ Letöltés...")
    files.download('/content/generated_images.zip')
    
    print("\n✓ Kész!")
else:
    print("❌ Nincs kép!")

---

## 💡 Hogyan Működik?

1. **TXT fájlok** → **Wildcard fájlok** (`/content/Fooocus/wildcards/`)
2. **Fooocus indítása** háttérben Gradio interface-szel
3. **Szerver ellenőrzés** - Vár, amíg a szerver elindul (max 5 perc)
4. **Gradio API** használata a generáláshoz
5. Prompt: `__wildcard_név__` → Fooocus random választ a listából
6. **Batch-enként** (max 32) generálás

## 🎯 Előnyök

- ✅ **Működik** - Nincs args_manager konfliktus
- ✅ **Automatikus** - Gradio API-n keresztül
- ✅ **Batch-elt** - 32 kép egyszerre
- ✅ **Testreszabható** - Performance, stílus, felbontás
- ✅ **Megbízható** - Ellenőrzi, hogy a szerver elindult-e

## 🐛 Hibaelhárítás

**"Connection refused" hiba:**
- Futtasd újra az 5. cellát (Fooocus indítása)
- Várj, amíg megjelenik a "✓ Fooocus elindult!" üzenet

**"Timeout" hiba:**
- A Colab GPU lassú lehet
- Próbáld újra futtatni az 5. cellát
- Ellenőrizd a Runtime → Change runtime type → GPU beállítást

---

**GitHub**: [dwick90/Fooocus_txt_prompts](https://github.com/dwick90/Fooocus_txt_prompts)